In [ ]:
from hypernodes import Pipeline, node


@node(output_name="doubled")
def double(x: int) -> int:
    return x * 2


@node(output_name="result")
def add_one(doubled: int) -> int:
    return doubled + 1


pipeline = Pipeline(nodes=[double, add_one], name="double_and_add")

In [2]:
result = pipeline.run(inputs={"x": 5})
result

{'doubled': 10, 'result': 11}

In [3]:
result = pipeline.map(inputs={"x": [1, 2, 3]}, map_over="x")
result

{'doubled': [2, 4, 6], 'result': [3, 5, 7]}

In [ ]:
import time

from hypernodes import Pipeline, node
from hypernodes.telemetry import ProgressCallback


@node(output_name="doubled")
def double(x: int) -> int:
    time.sleep(0.1)  # Small delay to see progress
    return x * 2


@node(output_name="result")
def add_one(doubled: int) -> int:
    time.sleep(0.1)
    return doubled + 1


# Create pipeline with progress callback
progress = ProgressCallback(enable=True)  # Disable for tests
pipeline = Pipeline(
    nodes=[double, add_one], callbacks=[progress], name="double_and_add"
)

result = pipeline.run(inputs={"x": 5})

Output()

├─ double ✓ (0.11s):  100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/1  [ 0:00:00 < 0:00:00 , ? it/s ]

├─ add_one ✓ (0.11s):  100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/1  [ 0:00:00 < 0:00:00 , ? it/s ]

In [ ]:
import logfire

from hypernodes import Pipeline, node
from hypernodes.telemetry import TelemetryCallback

# Configure logfire (local only, no cloud export)
logfire.configure(send_to_logfire=False)
telemetry = TelemetryCallback()

pipeline = Pipeline(
    nodes=[double, add_one], callbacks=[telemetry], name="double_and_add"
)

result = pipeline.run(inputs={"x": 5})

11:22:24.242 pipeline:pipeline_4392207568
11:22:24.243   node:double
11:22:24.345   node:add_one


In [11]:
chart = telemetry.get_waterfall_chart()
chart